<img src="https://bioinf.nl/~davelangers/hanze.png" align="right" />

# <span id="0">Casus *Hidden Markov Model* - Deel IV</span>

Inhoud:

* **<a href="#1">Het forward algoritme</a>**

* **<a href="#2">Rekenvoorbeeld</a>**

* **<a href="#3">Je eigen `HiddenMarkovModel` class</a>**

* **<a href="#4">CpG-eilandjes</a>**

* **<a href="#5">Afsluiting</a>**

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import numpy as np

<a id="1" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Het forward algoritme

In deel III van deze casus berekenden we de meest waarschijnlijke reeks toestanden om tot een waargenomen reeks emissies te leiden. Hiertoe gebruikten we het Viterbi-algoritme. In dit deel bekijken we een nauw gerelateerd probleem dat een zeer vergelijkbare aanpak heeft. Dit keer berekenen we de waarschijnlijk van een gegeven reeks observaties. In deel II berekenden we ook al een dergelijke waarschijnlijkheid, mits we de bijbehorende reeks toestanden kennen; we breiden dit nu uit naar de realistischere situatie waar de toestanden niet bekend zijn. De methode om dit te doen staat bekend als het *forward algoritme*.

Bekijk de onderstaande video over het forward algoritme. De benadering in deze video is iets anders dan die in de les. Noteer zaken die je noemenswaardig vindt om te onthouden.

In [ ]:
%%html
<iframe width="640" height="360" src="https://www.youtube.com/embed/9-sPm4CfcD0" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [ ]:
# UITWERKING

## Aantekeningen Forward Algoritme
Het forward algoritme berekent hoe waarschijnlijk een sequentie van waarnemingen is, gegeven de verborgen staten en kansen

Bij het algoritme wordt het probleem onderverdeeld in sub problemen, hiervoor kan een binaire boomweergave worden gebruikt. Bovenaan staat de reeks waarnemingen en in de eerste rij daaronder staan (kansen op) de staten op de laatste plaats in de sequentie, in de rij daaronder staan de (kansen op) de staten op de een-na-laatste plaats in de sequentie enzovoorts. Bij dit algoritme geldt dat de parent-nodes kunnen worden berekend door de child nodes bij elkaar op te tellen.

Bepaalde uitkomsten van berekeningen worden bijgehouden, namelijk die van de child nodes. Deze berekeningen zouden anders meermaals uitgevoerd moeten worden, wat het algoritme minder efficient zou maken. Dit is onderdeel van 'dynamic programming'. De basecase zijn de linker leaf nodes, waarvoor zijn de leave nodes, waarvoor geldt a1(Xi) P = (Xi) P(Y0|Xi). Dit is de eerste plek in de sequentie en hier wordt alleen gekeken naar de startkans en niet naar overgangskansen.

De kans voor staat van X kan op elke locatie t worden bepaald met:
$$
\alpha_{t}(X_i)
= \alpha_{t-1}(X_0)\,P(X_i \mid X_0)\,P(Y_t \mid X_i)
+ \alpha_{t-1}(X_1)\,P(X_i \mid X_1)\,P(Y_t \mid X_i)
+ \alpha_{t-1}(X_2)\,P(X_i \mid X_2)\,P(Y_t \mid X_i)
$$
Dit moet wordt herhaald voor het aantal staten dat er zijn.

<a id="2" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Rekenvoorbeeld

We bekijken voor de laatste keer het experiment met de tafels en knikkers. De gegevens omtrent overgangswaarschijnlijkheden en emissiekansen herhalen we hier:

| Tafel: |  ❶  |  ❷  |  ❸  |
| -----: | :-: | :-: | :-: |
| **Grabbelton:** | 6x blauw | 2x blauw | 1x blauw |
|                 | 3x geel  | 6x geel  | 0x geel  | 
|                 | 1x groen | 2x groen | 6x groen |
|                 | 2x rood  | 2x rood  | 5x rood  |
| **Dobbelsteen:** | ⚀→① | ⚀→① | ⚀→① |
|                  | ⚁→② | ⚁→② | ⚁→① |
|                  | ⚂→② | ⚂→② | ⚂→① |
|                  | ⚃→② | ⚃→③ | ⚃→① |
|                  | ⚄→③ | ⚄→③ | ⚄→② |
|                  | ⚅→③ | ⚅→③ | ⚅→③ |

Dit keer willen we de kans bepalen dat je de volgende reeks waarnemingen zou doen:

| **Beurt:** | 1     | 2     | 3     | 4     | 5     |
| ---------: | :---: | :---: | :---: | :---: | :---: |
| **Kleur:** | geel  | groen | blauw | rood  | groen |

Merk op dat dit niet hetzelfde is als de kans om deze emissies waar te nemen in combinatie met de meest waarschijnlijke reeks toestanden. Immers, er zijn ook andere, minder waarschijnlijke, reeksen toestanden die toch wel tot deze reeks emissies kunnen leiden. (Waarschijnlijk heb je al gemerkt dat je eigen reeks tafels niet exact gelijk was aan de meest waarschijnlijke reeks tafels, maar toch jouw emissies opleverde.)

De aanpak verloopt parallel aan die in deel III. Het forward algoritme begint weer met het begin van de reeks: eerst kijken we alleen naar beurt 1.

##### Beurt 1

In beurt 1 werd een gele knikker getrokken. Er zijn drie manieren hoe dit zou kunnen geschieden:

* Vanuit tafel ❶:<br />de kans dat je begint aan tafel ❶ is $\frac{1}{3}$, en de kans dat je daar een gele knikker trekt is $\frac{3}{12} = \frac{1}{4}$, dus de kans dat je via tafel ❶ een gele knikker trekt is

$$
P(❶_1 \cap \text{geel}_1) = \frac{1}{3} \cdot \frac{1}{4} = \frac{1}{12}
$$

* Vanuit tafel ❷:<br />de kans dat je begint aan tafel ❷ is $\frac{1}{3}$, en de kans dat je daar een gele knikker trekt is $\frac{6}{12} = \frac{1}{2}$, dus de kans dat je via tafel ❷ een gele knikker trekt is

$$
P(❷_1 \cap \text{geel}_1) = \frac{1}{3} \cdot \frac{1}{2} = \frac{1}{6}
$$

* Vanuit tafel ❸:<br />de kans dat je begint aan tafel ❸ is $\frac{1}{3}$, en de kans dat je daar een gele knikker trekt is $\frac{0}{12} = 0$, dus de kans dat je via tafel ❸ een gele knikker trekt is

$$
P(❸_1 \cap \text{geel}_1) = \frac{1}{3} \cdot 0 = 0
$$

Als je nu de kans wil weten dat je in beurt 1 een gele knikker trekt dien je deze drie waarden op te tellen. We kunnen dit wederom samenvatten in een tabel:

| **Beurt:** | 1     |
| ---------: | :---: |
|            | p = $\frac{1}{12}$ via $❶_1$ |
|            | p = $\frac{1}{6}$ via $❷_1$ |
|            | p = $0$ via $❸_1$ |

De *totale kans* is

$$
p_\text{totaal} = \frac{1}{12} + \frac{1}{6} + 0 = \frac{1}{4}
$$

Dit is niet vreemd omdat er vier kleuren waren die allemaal even vaak voorkwamen over alle tafels heen opgeteld. Als ik geen idee heb aan welke tafel ik zat en ik blijk één van de vier kleuren te hebben getrokken die allemaal even waarschijnlijk zijn, dan moet die kans dus wel $p = \frac{1}{4}$ bedragen.

Tot dusverre was de uitkomst in essentie hetzelfde als in deel III. We kijken hier alleen naar de *totale kans* in plaats van de *meest waarschijnlijke reeks toestanden*. We gaan nu verder met beurt 2 waar de resultaten gaan verschillen.

##### Beurt 2

In beurt 2 werd een groene knikker getrokken. Er zijn drie manieren hoe dit zou kunnen geschieden:

* Vanuit tafel ❶:<br />Je kan op drie manieren aan tafel ❶ terechtkomen in beurt 2:
  * als je aan tafel ❶ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❶ was $\frac{1}{12}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❶ met een dobbelsteenworp naar tafel ❶ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❶ een groene knikker trekt is $\frac{1}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❶ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{12} \cdot \frac{1}{6} \cdot \frac{1}{12} = \frac{1}{864} \approx 0.00116$
  * als je aan tafel ❷ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❷ was $\frac{1}{6}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❷ met een dobbelsteenworp naar tafel ❶ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❶ een groene knikker trekt is $\frac{1}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{6} \cdot \frac{1}{6} \cdot \frac{1}{12} = \frac{1}{432} \approx 0.00231$
  * als je aan tafel ❸ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❸ was $0$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❸ met een dobbelsteenworp naar tafel ❶ gaat is $\frac{4}{6}$;
    * de kans dat je vanuit tafel ❶ een groene knikker trekt is $\frac{1}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $0 \cdot \frac{4}{6} \cdot \frac{1}{12} = 0 = 0.00000$

De som van al deze mogelijkheden is

$$
p = \frac{1}{864} + \frac{1}{432} + 0 = \frac{1}{288} \approx 0.00347
$$

* Vanuit tafel ❷:<br />Je kan op drie manieren aan tafel ❷ terechtkomen in beurt 2:
  * als je aan tafel ❶ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❶ was $\frac{1}{12}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❶ met een dobbelsteenworp naar tafel ❷ gaat is $\frac{3}{6}$;
    * de kans dat je vanuit tafel ❷ een groene knikker trekt is $\frac{2}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❶ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{12} \cdot \frac{3}{6} \cdot \frac{2}{12} = \frac{1}{144} \approx 0.00694$
  * als je aan tafel ❷ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❷ was $\frac{1}{6}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❷ met een dobbelsteenworp naar tafel ❷ gaat is $\frac{2}{6}$;
    * de kans dat je vanuit tafel ❷ een groene knikker trekt is $\frac{2}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{6} \cdot \frac{2}{6} \cdot \frac{2}{12} = \frac{1}{108} \approx 0.00926$
  * als je aan tafel ❸ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❸ was $0$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❸ met een dobbelsteenworp naar tafel ❷ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❷ een groene knikker trekt is $\frac{2}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❸ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $0 \cdot \frac{1}{6} \cdot \frac{2}{12} = 0 = 0.00000$

De som van al deze mogelijkheden is

$$
p = \frac{1}{144} + \frac{1}{108} + 0 = \frac{7}{432} \approx 0.01620
$$

* Vanuit tafel ❸:<br />Je kan op drie manieren aan tafel ❸ terechtkomen in beurt 2:
  * als je aan tafel ❶ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❶ was $\frac{1}{12}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❶ met een dobbelsteenworp naar tafel ❸ gaat is $\frac{2}{6}$;
    * de kans dat je vanuit tafel ❸ een groene knikker trekt is $\frac{6}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❶ in beurt 1 naar tafel ❸ in beurt 2 een groene knikker trekt is $\frac{1}{12} \cdot \frac{2}{6} \cdot \frac{6}{12} = \frac{1}{72} \approx 0.01389$
  * als je aan tafel ❷ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❷ was $\frac{1}{6}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❷ met een dobbelsteenworp naar tafel ❸ gaat is $\frac{3}{6}$;
    * de kans dat je vanuit tafel ❸ een groene knikker trekt is $\frac{6}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❸ in beurt 2 een groene knikker trekt is $\frac{1}{6} \cdot \frac{3}{6} \cdot \frac{6}{12} = \frac{1}{24} \approx 0.04167$
  * als je aan tafel ❸ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❸ was $0$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❸ met een dobbelsteenworp naar tafel ❸ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❸ een groene knikker trekt is $\frac{6}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❸ in beurt 1 naar tafel ❸ in beurt 2 een groene knikker trekt is $0 \cdot \frac{1}{6} \cdot \frac{6}{12} = 0 = 0.00000$

De som van al deze mogelijkheden is

$$
p = \frac{1}{72} + \frac{1}{24} + 0 = \frac{1}{18} \approx 0.05556
$$

We breiden de tabel uit met deze drie resultaten:

| **Beurt:** | 1     | 2     |
| ---------: | :---: | :---: |
|            | p = $\frac{1}{12}$ via $❶_1$ | p = 0.00347 via $❶_2$ |
|            | p = $\frac{1}{6}$ via $❷_1$ | p = 0.01620 via $❷_2$ |
|            | p = $0$ via $❸_1$ | p = 0.05556 via $❸_2$ |

Als we nu alles tezamen bekijken, is de totale kans gelijk aan

$$
p_\text{totaal} \approx 0.00347 + 0.01620 + 0.05556 = 0.07523
$$

Als de kans op een gele knikker (in beurt 1) en een groene knikker (in beurt 2) onafhankelijk waren geweest, dan hadden we verwacht dat de kans op deze twee waarnemingen gelijk zou zijn aan $p = \frac{1}{4} \cdot \frac{1}{4} = \frac{1}{16} \approx 0.0625$ (wederom omdat elk van de vier kleuren over alle tafels tezamen even vaak voorkomen). De echte kans zoals we die zojuist berekenden blijkt hoger te zijn, dus een gele knikker wordt waarschijnlijker gevolgd door een groene knikker. Ze zijn dus *niet* onafhankelijk.

##### Enzovoorts

We voeren steeds hetzelfde recept uit. Voor elke mogelijke tafel in de huidige beurt, beschouw elke mogelijke tafel uit de vorige beurt. Vermenigvuldig de kans die je uit die vorige beurt voor die tafel berekend had met de kans om van de vorige naar de huidige tafel over te gaan, en met de kans om dan de huidige kleur knikker te trekken. De nieuwe kans is gelijk aan de som van alle dergelijke kansen over alle vorige tafels.

Ga na dat je de volgende resultaten krijgt:

| **Beurt:** | 1     | 2     | 3     | 4     | 5     |
| ---------: | :---: | :---: | :---: | :---: | :---: |
|            | p = $\frac{1}{12}$ via $❶_1$ | p = 0.00347 via $❶_2$ | p = 0.02016 via $❶_3$ | p = 0.00081 via $❶_4$ | p = 0.00023 via $❶_5$ |
|            | p = $\frac{1}{6}$ via $❷_1$ | p = 0.01620 via $❷_2$ | p = 0.00273 via $❷_3$ | p = 0.00187 via $❷_4$ | p = 0.00027 via $❷_5$ |
|            | p = $0$ via $❸_1$ | p = 0.05556 via $❸_2$ | p = 0.00154 via $❸_3$ | p = 0.00348 via $❸_4$ | p = 0.00089 via $❸_5$ |

De eindconclusie lezen we nu af in de laatste kolom. Uiteindelijk moet je bij een van de drie tafels uitkomen in beurt 5, en we weten voor elke tafel wat dan de kans op de getrokken kleuren is. Dus de totale kans op de waargenomen emissies is gelijk aan

$$
p_\text{totaal} \approx 0.00023 + 0.00027 + 0.00089  = 0.00139
$$

Merk op dat zowel de berekende kans $p = 0.000054$ uit deel II (in combinatie met de daadwerkelijke reeks tafels) als de berekende kans $p = 0.00029$ uit deel III (in combinatie met de meest waarschijnlijke reeks tafels) lager moeten uitvallen dan deze kans $p = 0.00139$ uit deel IV (in combinatie met alle mogelijke reeksen tafels).

Bepaal nu voor de eerste drie beurten uit jouw eigen experimentele waarnemingen wat de kans is dat je deze emissies zou waarnemen.

**Berekening Yamila:**

| **Beurt:** |              1               |               2                |                3                 |
| ---------: | :--------------------------: | :----------------------------: | :------------------------------: |
|     Kleur: |             Geel             |             Groen              |               Rood               |
|            | p = $\frac{1}{12}$ via $❶_1$ | p = $\frac{1}{288}$  via $❶_2$ |  p = $\frac{209}{31104}$ $❶_3$   |
|            | p = $\frac{1}{6}$ via $❷_1$  | p = $\frac{7}{432}$ via $❷_2$  | p = $\frac{85}{31104}$ via $❷_3$ |
|            |      p = $0$ via $❸_1$       | p = $\frac{1}{18}$  via $❸_2$  |  p = $\frac{5}{648}$ via $❸_3$   |

**Eindconclusie**
De totale kans op de waargenomen emissies is gelijk aan:

$$
p_\text{totaal} = \frac{209}{31104} \frac{85}{31104} + \frac{5}{648} = \frac{89}{5184} \approx 0.01717
$$

---

**Berekening Mirte:**

| **Beurt:** | 1 | 2 | 3 |
| ---------: | :---: | :---: | :---: |
|Kleur|Groen|Rood|Rood|
|            | p = $\frac{1}{18}$ via $❶_1$ | p = $\frac{1}{54}$ via $❶_2$ | p = $\frac{7}{1728}$ via $❶_3$ |
|            | p = $\frac{1}{18}$ via $❷_1$ | p = $\frac{5}{432}$ via $❷_2$ | p = $\frac{31}{10368}$ via $❷_3$ |
|            | p = $\frac{5}{36}$ via $❸_1$ | p = $\frac{25}{864}$ via $❸_2$ | p = $\frac{145}{20736}$ via $❸_3$ |

De totale kans op de waargenomen emissies is:

$$
p_\text{totaal} = \frac{7}{1728} + \frac{31}{10368} + \frac{145}{20736} = \frac{97}{6912} \approx 0.0140
$$


<a id="3" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Je eigen `HiddenMarkovModel` class

In deel II van deze casus schreef je een methode `score()` die twee argumenten ontvangt (emissies en toestanden). Deze berekent de waarschijnlijkheid van een gegeven reeks emissies in combinatie met een gegeven reeks toestanden. In de praktijk zijn toestanden niet meetbaar. Daarom zou je graag de waarschijnlijkheid kunnen berekenen van de gegeven reeks emissies op zich. Deze is gelijk aan de som van de waarschijnlijkheden over alle mogelijke reeksen van toestanden.

$$
P(\text{ emissies }) = \sum_{\text{ toestanden } T} P(\text{ emissies } \cap T)
$$

In deel II werd dit gedemonstreerd door expliciet te sommeren over alle reeksen toestanden. Dat kan wanneer het aantal mogelijke toestandsreeksen behapbaar is. Echter, dit wordt al gauw heel onpraktisch omdat het aantal mogelijke reeksen toestanden exponentieel toeneemt naarmate de lengte van de reeks toeneemt. Zojuist hebben we echter het *forward algoritme* bekeken dat dit in lineaire tijdcomplexiteit kan, wat het prima uitvoerbaar maakt.

Pas de `score()` methode van je eigen `HiddenMarkovModel` klasse aan zodat het tweede argument (d.w.z. de toestanden) optioneel zijn. Als de gebruiker toestanden meegeeft, dan dient je methode de log-waarschijnlijkheid op de emissies en toestanden samen te berekenen, zoals dat al in deel II gebeurde. Dat is dus niet nieuw. Echter, voeg functionaliteit toe die wordt uitgevoerd als er geen toestanden worden meegegeven en die in dat geval de log-waarschijnlijkheid van de emissies in hun eentje berekent met het forward algoritme. Je krijgt dus iets als volgt.

```python
class HiddenMarkovModel():
    ...
    def score(self, X, state_sequence=None):
        if state_sequence is None:
            # Implementatie forward-algoritme
        else:
            # Implementatie uit deel II
        return log_prob
    ...
```

Gebruik je eigen module om de berekening uit de voorgaande oefening te controleren. Komt er hetzelfde antwoord uit? Bereken ook de log-waarschijnlijkheid voor je hele reeks eigen waarnemingen, en voor de hele reeks waarnemingen van de hele klas; deze berekeningen zijn wat omslachtig om met de hand te doen, maar je klasse zou het met gemak aan moeten kunnen.

Vergelijk tenslotte je uitkomsten met wat `CategoricalHMM(...).score(X)` uit de `hmmlearn` module berekent.

In [4]:
# UITWERKING
from hmmmodel import HiddenMarkovModel as HMM
import numpy as np

# Startkansen (gelijk voor elke tafel)
startprob = np.array([1 / 3, 1 / 3, 1 / 3])

# Overgangskansen
transprob = {
    0: [1 / 6, 1 / 2, 1 / 3],
    1: [1 / 6, 1 / 3, 1 / 2],
    2: [2 / 3, 1 / 6, 1 / 6]
}

# Emissiekansen
emissionprob = {
    0: [1 / 6, 1 / 12, 1 / 4, 1 / 2],
    1: [1 / 6, 1 / 6, 1 / 2, 1 / 6],
    2: [5 / 12, 1 / 2, 0, 1 / 12]
}

states = [0, 1, 2]  #tafel 1 = 0, tafel 2 = 1, tafel 3 = 2
emissions = [0, 1, 2, 3]  # Blue = 0, Green = 1, Yellow = 2, Red = 3

model_HMM = HMM(startprob, transprob, emissionprob, states, emissions)

# Vergelijking met hmmlearn CategoricalHMM
from hmmlearn import hmm
model_hmm = hmm.CategoricalHMM(n_components=3, n_features=4, init_params="")
model_hmm.transmat_ = np.array(list(transprob.values()))
model_hmm.emissionprob_ = np.array(list(emissionprob.values()))
model_hmm.startprob_ = startprob


# Controleren voorgaande oefening
emissions_yamila_3 = [2, 1, 3]
emissions_mirte_3 = [2, 3, 3]


log_probability_mirte_3 = model_hmm.score(np.array(emissions_mirte_3).reshape(-1, 1))
log_probability_yamila_3 = model_hmm.score(np.array(emissions_yamila_3).reshape(-1, 1))

print(f'Predicted probability of observations using our model: ln(p) = {round(model_HMM.score(emissions_yamila_3), 3)} and using hmmlearn: ln(p) = {round(log_probability_yamila_3, 3)}')

print(f'Predicted probability of observations using our model: ln(p) = {round(model_HMM.score(emissions_mirte_3), 3)} and using hmmlearn: ln(p) = {round(log_probability_mirte_3, 3)}')


Predicted probability of observations using our model: ln(p) = -3.712 and using hmmlearn: ln(p) = -3.712
Predicted probability of observations using our model: ln(p) = -4.561 and using hmmlearn: ln(p) = -4.561


In [5]:
# Berekenen log waarschijnlijkheid eigen reeks
emissions_yamila = [2, 1, 3, 1, 0, 1, 3, 1, 0, 3, 0, 1, 2, 2, 0, 0, 2, 1, 0, 2, 2, 0, 0, 2, 0, 2, 3, 2, 2, 2, 0]
emissions_mirte = [2, 3, 3, 0, 1, 2, 0, 2, 3, 2, 2, 0, 2, 0, 1, 2, 2, 0, 3, 0, 1, 3, 1, 2, 0, 0, 2, 0, 1]

log_probability_yamila = model_hmm.score(np.array(emissions_yamila).reshape(-1, 1))
log_probability_mirte = model_hmm.score(np.array(emissions_mirte).reshape(-1, 1))

print(f'Predicted probability of observations using our model: ln(p) = {round(model_HMM.score(emissions_yamila), 3)} and using hmmlearn: ln(p) = {round(log_probability_yamila, 3)}')

print(f'Predicted probability of observations using our model: ln(p) = {round(model_HMM.score(emissions_mirte), 3)} and using hmmlearn: ln(p) = {round(log_probability_mirte, 3)}')

Predicted probability of observations using our model: ln(p) = -42.468 and using hmmlearn: ln(p) = -42.468
Predicted probability of observations using our model: ln(p) = -40.496 and using hmmlearn: ln(p) = -40.496


In [6]:
# Berekenen log waarschijnlijkheid waarnemingen van de hele klas
all_emissions = [
    1,2,2,1,3,1,2,0,3,2,3,1,2,3,1,1,2,0,2,3,1,2,0,0,3,0,0,3,1,3,1,2,0,0,1,0,1,3,0,2,
    3,1,3,1,3,2,1,3,3,1,2,0,3,2,1,2,3,1,0,0,2,1,0,3,1,2,3,2,0,2,3,2,0,3,0,2,1,1,0,0,
    1,2,0,1,1,0,0,1,0,1,3,1,1,1,0,2,1,1,3,0,3,0,3,3,0,2,2,1,1,2,0,1,2,3,1,3,3,2,0,0,
    2,3,0,3,0,2,3,1,3,3,3,1,0,0,3,2,2,1,1,3,0,3,0,3,1,2,1,2,2,2,2,0,2,3,1,1,2,0,2,3,
    3,0,1,2,0,2,3,2,2,0,2,0,1,2,2,0,3,0,1,3,1,2,0,0,2,0,1,1,1,2,1,2,0,2,0,2,1,0,2,1,
    2,0,0,3,3,2,2,3,3,2,0,1,0,3,1,0,1,2,0,0,3,2,2,0,2,0,3,2,3,2,1,1,2,2,2,0,0,0,2,0,
    0,3,0,0,2,3,3,1,1,2,3,3,2,0,1,1,2,1,1,1,3,0,1,2,0,1,3,1,2,3,2,2,3,1,2,2,1,3,3,2,
    3,3,0,1,2,0,0,1,1,3,0,0,1,1,2,0,2,0,2,3,1,1,1,3,3,3,3,3,3,2,1,1,1,2,2,1,0,3,0,3,
    1,2,2,3,2,2,2,3,0,2,3,1,3,2,1,0,1,2,1,3,2,0,1,2,0,0,3,1,2,1,1,0,3,1,2,1,2,2,0,0,
    1,3,3
    ]

log_probability_all = model_hmm.score(np.array(all_emissions).reshape(-1, 1))

print(f'Predicted probability of observations using our model: ln(p) = {round(model_HMM.score(all_emissions), 3)} and using hmmlearn: ln(p) = {round(log_probability_all, 3)}')


Predicted probability of observations using our model: ln(p) = -506.108 and using hmmlearn: ln(p) = -506.108


De uitkomsten lijken voor alle berekeningen gelijk te zijn voor de score methode van ons model en dat van hmmlearn. De uitkomst van de eerste 3 van de sequentie van Yamila lijken wel te verschillen met dat wat met de hand berekend is. Maar omdat dit overeenkomt bij zowel ons model als dat van hmmlearn kan ervan uit worden gegaan dat er een foutje zit in de berekening met de hand.

<a id="4" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### CpG-eilandjes

Op de website [CpG Educate](https://cpgeducate.com/cpg.htm) wordt een Hidden Markov Model voorgesteld om CpG-eilandjes te beschrijven. Het omvat 8 toestanden, die min of meer - maar niet exact - overeenkomen met de nucleotiden A, C, G, en T, zowel binnen (`+`) als buiten (`-`) CpG-eilandjes. De overgangswaarschijnlijkheden, emissiekansen, en begintoestandsverdeling worden daar als volgt gekozen:

| Transities  | A (`+`) | C (`+`) | G (`+`) | T (`+`) | A (`-`) | C (`-`) | G (`-`) | T (`-`) |
| :---------: | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: |
| **A** (`+`) | 0.2233  | 0.2020  | 0.3106  | 0.1745  | 0.0229  | 0.0377  | 0.0164  | 0.0196  |
| **C** (`+`) | 0.2667  | 0.2511  | 0.0746  | 0.3138  | 0.0230  | 0.0465  | 0.0087  | 0.0226  |
| **G** (`+`) | 0.1846  | 0.2703  | 0.2326  | 0.1815  | 0.0399  | 0.0272  | 0.0531  | 0.0178  |
| **T** (`+`) | 0.0930  | 0.2520  | 0.3459  | 0.2450  | 0.0093  | 0.0184  | 0.0182  | 0.0251  |
| **A** (`-`) | 0.0323  | 0.0154  | 0.0342  | 0.0154  | 0.2684  | 0.1679  | 0.3238  | 0.1496  |
| **C** (`-`) | 0.0243  | 0.0392  | 0.0089  | 0.0146  | 0.3052  | 0.2486  | 0.0650  | 0.3013  |
| **G** (`-`) | 0.0480  | 0.0303  | 0.0287  | 0.0187  | 0.2575  | 0.2070  | 0.2432  | 0.1737  |
| **T** (`-`) | 0.0210  | 0.0274  | 0.0372  | 0.0209  | 0.0966  | 0.2415  | 0.3511  | 0.2114  |

| Emissies    |   A    |   C    |   G    |   T    |
| :---------: | :----: | :----: | :----: | :----: |
| **A** (`+`) | 0.8358 | 0.0607 | 0.0564 | 0.0501 |
| **C** (`+`) | 0.0382 | 0.9006 | 0.0376 | 0.0266 |
| **G** (`+`) | 0.0344 | 0.0451 | 0.8904 | 0.0331 |
| **T** (`+`) | 0.0624 | 0.0486 | 0.0544 | 0.8376 |
| **A** (`-`) | 0.8699 | 0.0434 | 0.0445 | 0.0452 |
| **C** (`-`) | 0.0499 | 0.8827 | 0.0422 | 0.0283 |
| **G** (`-`) | 0.0382 | 0.0307 | 0.8935 | 0.0406 |
| **T** (`-`) | 0.0729 | 0.0389 | 0.0360 | 0.8552 |

| Startkansen |   p    |
| :---------: | :----: |
| **A** (`+`) | 0.0742 |
| **C** (`+`) | 0.1005 |
| **G** (`+`) | 0.1181 |
| **T** (`+`) | 0.0309 |
| **A** (`-`) | 0.0770 |
| **C** (`-`) | 0.2492 |
| **G** (`-`) | 0.2877 |
| **T** (`-`) | 0.0692 |

Merk op dat door afrondfoutjes de kansen niet exact optellen tot 100%, maar dit kun je zonodig oplossen door de getallen te schalen.

Initialiseer met behulp van je eigen module een Hidden Markov Model met de bovenstaande eigenschappen. Gebruik dit om de sequentie uit het meegeleverde fasta-bestand `Casus_HiddenMarkovModel.fasta` te analyseren en te bepalen welke delen van de sequentie CpG-eilandjes vormen. Komen de resultaten overeen met die van de website (zie het tabblad *Decoding*)? De output van de website vermeldt onderaan ook een log-waarschijnlijkheid. Komt deze overeen met je eigen berekeningen?

In deel III pasten we een eenvoudiger Hidden Markov Model toe met de volgende kenmerken.

| Emissies          |   A   |   C   |   G   |   T   |
| :---------------: | :---: | :---: | :---: | :---: |
| **CGI** (`+`)     | 0.20  | 0.30  | 0.30  | 0.20  |
| **non-CGI** (`-`) | 0.25  | 0.25  | 0.25  | 0.25  |

| Transities        | CGI (`+`)     | non-CGI (`-`) |
| :---------------: | :-----------: | :-----------: |
| **CGI** (`+`)     |    0.990      |    0.010      |
| **non-CGI** (`-`) |    0.001      |    0.999      |

Vergelijk de log-waarschijnlijkheid die het eerste model met acht toestanden aan de waargenomen sequentie toekent met de log-waarschijnlijkheid die dit tweede model met twee toestanden toekent. Bereken of schat het ratio tussen de beide kansen. Is de sequentie volgens het ene model veel meer of minder waarschijnlijk dan volgens het andere, of maakt het niet heel veel uit? Heb je reden te vermoeden dat een van beide modellen beter is dan het andere?

Dit laatste illustreert de belangrijkste reden om dergelijke kansen op emissies te berekenen: je kunt iets zeggen over "hoe goed een reeks emissies bij het model past". En dus ook omgekeerd, "hoe goed een model bij een reeks emissies past". En dit stelt je in principe in staat om tussen verschillende versies van modellen het beste te kiezen, of om een model te optimaliseren voor gegeven data. Dit komt neer op het *trainen* van een Hidden Markov Model. We gaan hier nu niet dieper op in.

In [ ]:
# UITWERKING

In [7]:
# Model met eigenschappen gebaseerd op de versie van CpG educate
from hmmmodel import HiddenMarkovModel as HMM
import pandas as pd
import numpy as np

sequence = ""

with open("Casus_HiddenMarkovModel.fasta") as f:
    next(f)
    for line in f:
        sequence += line.strip()

def encode_sequence(seq):
    mapping = {'A': 0, 'C': 1, 'G' : 2, 'T' : 3}
    return [mapping[n]for n in seq]


encoded_sequence = encode_sequence(sequence)

startprob = np.array([0.0742, 0.1005, 0.1181, 0.0309, 0.0770, 0.2492, 0.2877, 0.0692])

emissionprob = np.array([
    [0.8358, 0.0607, 0.0564, 0.0501],
    [0.0382, 0.9006, 0.0376, 0.0266],
    [0.0344, 0.0451, 0.8904, 0.0311],
    [0.0624, 0.0486, 0.0544, 0.8376],
    [0.8699, 0.0434, 0.0445, 0.0452],
    [0.0499, 0.8827, 0.0422, 0.0283],
    [0.0382, 0.0307, 0.8935, 0.0406],
    [0.0729, 0.0389, 0.0360, 0.8552]
])

transprob = np.array([
    [0.2233, 0.2020, 0.3106, 0.1745, 0.0229, 0.0377, 0.0164, 0.0196],
    [0.2667, 0.2511, 0.0746, 0.3138, 0.0230, 0.0465, 0.0087, 0.0226],
    [0.1846, 0.2703, 0.2326, 0.1815, 0.0399, 0.0272, 0.0531, 0.0178],
    [0.0930, 0.2520, 0.3459, 0.2450, 0.0093, 0.0184, 0.0182, 0.0251],
    [0.0323, 0.0154, 0.0342, 0.0154, 0.2684, 0.1679, 0.3238, 0.1496],
    [0.00243, 0.0392, 0.0089, 0.0146, 0.3052, 0.2486, 0.0650, 0.3013],
    [0.0480, 0.0303, 0.0287, 0.0187, 0.2572, 0.2070, 0.2432, 0.1737],
    [0.0210, 0.0274, 0.0372, 0.0209, 0.0966, 0.2415, 0.3511, 0.2114]
])

states = [0, 1, 2, 3, 4, 5, 6, 7] # 0 = A+, 1 = C+, 2 = G+, 3 = T+, 4 = A-, 5 = C-, 6 = G-, 7 = T-
emissions = [0, 1, 2, 3]  # 0 = A, 1 = C, 2 = G, 3 = T
model = HMM(startprob, transprob, emissionprob, states, emissions)
scored = model.score(encoded_sequence)
predicted = model.predict(encoded_sequence)
score_predicted = model.score(encoded_sequence, predicted)

print(f'Log-waarschijnlijkheid over alle staten: ln(p) {round(scored, 3)}')
print(f'Log-waarschijnlijkheid voor meest waarschijnlijke pad: ln(p) {round(score_predicted, 3)}')
print('''De log waarschijnlijkheid berekent door cpgeducate.com: ln(p) -116884.366
Dit zit dicht in de buurt van onze berekening maar komt niet helemaal overeen.''')

Log-waarschijnlijkheid over alle staten: ln(p) -98406.441
Log-waarschijnlijkheid voor meest waarschijnlijke pad: ln(p) -116887.231
De log waarschijnlijkheid berekent door cpgeducate.com: ln(p) -116884.366
Dit zit dicht in de buurt van onze berekening maar komt niet helemaal overeen.


In [8]:
# Eenvoudig model uit deel III
startprob = np.array([1/4, 1/4, 1/4, 1/4])

emissionprob = np.array([
    [0.20, 0.30, 0.30, 0.20],
    [0.25, 0.25, 0.25, 0.25]
])

transprob = np.array([
    [0.990, 0.010],
    [0.001, 0.999]
])

states = [0, 1]  # 0 = CGI, 1 = non-CGI
emissions = [0, 1, 2, 3]  # 0 = A, 1 = C, 2 = G, 3 = T
model = HMM(startprob, transprob, emissionprob, states, emissions)
scored = model.score(encoded_sequence)
predicted = model.predict(encoded_sequence)
score_predicted2 = model.score(encoded_sequence, predicted)

print(f'Log-waarschijnlijkheid over alle staten: ln(p) {round(scored, 3)}')
print(f'Log-waarschijnlijkheid voor meest waarschijnlijke pad: ln(p) {round(score_predicted, 3)}')
print('''Log-waarschijnlijkheid model met 8 toestanden: ln(p) -116887.231
Log-waarschijnlijkheid model met 2 toestanden: ln(p) -99753.273\n''')

log_ratio = score_predicted - score_predicted2
ratio = np.exp(log_ratio)

print(f'''De ratio is: {ratio}
Het model met 8 staten is iets waarschijnlijker maar de ratio is niet heel groot.
Het model met 2 staten is simpeler om te begrijpen en te implementeren.
Dit is een belangrijke overweging om te maken bij het kiezen welk model er gebruikt gaat worden.''')

Log-waarschijnlijkheid over alle staten: ln(p) -99674.657
Log-waarschijnlijkheid voor meest waarschijnlijke pad: ln(p) -116887.231
Log-waarschijnlijkheid model met 8 toestanden: ln(p) -116887.231
Log-waarschijnlijkheid model met 2 toestanden: ln(p) -99753.273

De ratio is: 0.0
Het model met 8 staten is iets waarschijnlijker maar de ratio is niet heel groot.
Het model met 2 staten is simpeler om te begrijpen en te implementeren.
Dit is een belangrijke overweging om te maken bij het kiezen welk model er gebruikt gaat worden.


<a id="5" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Afsluiting

In deze casus heb je de werking van Hidden Markov Modellen bestudeerd in de context van de detectie van CpG-eilandjes. Hierbij is aandacht besteed aan het *genereren* van sequenties, het *evalueren* van de waarschijnlijkheid ervan, en het *decoderen* van de toestanden behorende bij een reeks emissies. Aan een aantal andere zaken zijn we niet toe gekomen in deze introductie. Hieronder vallen voornamelijk het [forward-backward algoritme](https://en.wikipedia.org/wiki/Forward%E2%80%93backward_algorithm) voor het bepalen van de meest waarschijnlijke toestand op een specifieke positie in de reeks, of het [Baum-Welch algoritme](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) voor het trainen van de parameters van een Hidden Markov Model. Als je dit interessant lijkt zou je dit zelf nader kunnen onderzoeken wanneer je met je Verdiepend Model aan de slag gaat. Desalniettemin zou je nu een goed idee moeten hebben over wat een Hidden Markov Model is, hoe het werkt, en waarvoor je het zoal kan gebruiken.

Raina Robeva en Robin Davies van [Sweet Briar College](https://www.sbc.edu/) gaven tijdens de conferentie [Algebraic and discrete biological models for undergraduate courses](https://legacy.nimbios.org//tutorials/TT_mathbio) in juni 2014 een lezing aan het Amerikaanse [National Institute for Mathemathical and Biological Synthesis](https://www.nimbios.org/) (NIMBioS) met de titel **CpG island identification with hidden Markov models for DNA sequence analysis**. Deze lezing vat uitstekend samen waar deze casus over ging. Daarnaast wordt de biologische relevantie van Hidden Markov Models uitgebreider toegelicht en gemotiveerd. De lezing is online beschikbaar en hieronder ingevoegd.

Gefeliciteerd als je alle onderdelen hebt af weten te krijgen! Bekijk als "samenvatting" van deze casus deze lezing onder het genot van een hapje en een drankje.

In [ ]:
%%html
<iframe width="640" height="360" src="https://www.youtube.com/embed/K7T5ZY51Km0?si=ZlvO6ezFLFUWyCMY" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

***

&copy; 2025 - Dave R.M. Langers <d.r.m.langers@pl.hanze.nl>